In [1]:
from rlcube.models.models import DNN
from rlcube.envs.cube2 import Cube2Env
import numpy as np
import torch

net = DNN()
net.load("models/model_best.pth")
net.eval()

DNN(
  (fc_in): Linear(in_features=144, out_features=512, bias=True)
  (residual_blocks): ModuleList(
    (0-3): 4 x ResidualBlock(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=512, out_features=1024, bias=True)
      (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc2): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
  (fc_value): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (fc_policy): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=12, bias=True)
  )
)

In [9]:
batch_obs = []
env = Cube2Env()
for _ in range(10):
    obs, _, _, _, _ = env.step(env.action_space.sample())
    batch_obs.append(torch.tensor(obs, dtype=torch.float32))
batched_obs = torch.stack(batch_obs)
out = net(batched_obs)

for i in range(10):
    env = Cube2Env.from_obs(batch_obs[i])
    env.print_js_code()
    print(out["value"][i].item())
    print(torch.argmax(out["policy"][i]).item())
    print()

rotationController.setState([[0, 0, 4, 4], [1, 1, 5, 5], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 1, 1], [5, 5, 0, 0]]);
0.40487873554229736
4

rotationController.setState([[0, 4, 0, 4], [1, 1, 5, 5], [2, 5, 2, 0], [3, 4, 3, 1], [4, 2, 1, 2], [5, 3, 0, 3]]);
0.0839405208826065
7

rotationController.setState([[0, 4, 0, 4], [5, 1, 5, 1], [1, 5, 4, 0], [0, 4, 5, 1], [3, 2, 3, 2], [2, 3, 2, 3]]);
-0.23320673406124115
3

rotationController.setState([[0, 5, 0, 1], [5, 4, 5, 0], [1, 5, 4, 4], [0, 4, 1, 1], [3, 3, 2, 2], [2, 3, 2, 3]]);
0.31869572401046753
0

rotationController.setState([[5, 5, 1, 1], [4, 4, 0, 0], [5, 5, 4, 4], [0, 0, 1, 1], [3, 3, 2, 2], [3, 3, 2, 2]]);
-0.16905824840068817
7

rotationController.setState([[5, 4, 1, 4], [4, 1, 0, 1], [5, 5, 4, 0], [0, 0, 5, 1], [3, 2, 3, 2], [3, 3, 2, 2]]);
0.20266102254390717
3

rotationController.setState([[2, 3, 1, 4], [3, 3, 0, 1], [5, 5, 4, 0], [0, 1, 0, 5], [4, 1, 3, 2], [5, 4, 2, 2]]);
0.6111429333686829
3

rotationController.setState([[2, 0